# Transfer of Airport Movement Data

Notebook addressing the creation of an Excel file report for the "Transfer of Airport Movement Data" on Lommis Airfield.

Template used in accordance with the one provided by the Federal Office of Civil Aviation (BAZL): https://www.bazl.admin.ch/bazl/de/home/themen/geoinformation_statistik/statistik/statistische_datenlieferungen.html

In [ ]:
## IMPORT LIBRARIES ##
import os
import lommis_func
import warnings
import numpy as np
import shutil
from openpyxl import load_workbook
from openpyxl.styles import Alignment
from datetime import datetime, timedelta
from traffic.core import Traffic, Flight
from traffic.data import airports, opensky
warnings.simplefilter("ignore")

c:\Users\fusg\Documents\Lommis_App\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Year and Month of the fetching data
year = 2025
month = 1

In [ ]:
## FETCH DATA FROM LOMMIS' AIRFIELD ##

# Define the start date
start_date = datetime.strptime(f"{year}-{month}-01 00:00", "%Y-%m-%d %H:%M")
 
# Number of iterations
num_iterations = 10

# Define the parquet directory path
folder_name = f"{month:02d}{year}"
parquet_dir_path = os.path.join("Flights/", folder_name)

os.makedirs(parquet_dir_path, exist_ok=True)
# Delete all contents of the parquet directory if any exists
# if os.path.exists(parquet_dir_path):
#     shutil.rmtree(parquet_dir_path)
#     os.makedirs(parquet_dir_path)
#     print(f"Deleted all contents in the directory: {parquet_dir_path}")
 
for i in range(num_iterations):
    # Generate stop date
    stop_date = start_date + timedelta(days=1)
 
    # Format both dates as strings in the desired format
    start_str = start_date.strftime("%Y-%m-%d %H:%M")
    stop_str = stop_date.strftime("%Y-%m-%d %H:%M")
    print(f"Fetching data for Start: {start_str}, Stop: {stop_str}")
    
    # fetch data
    trajs = opensky.history(
        start = start_str,
        stop = stop_str,
        airport = "LSZT",
        selected_columns=(
        'time', 'icao24', 'callsign', 'lat', 'lon', 'heading', 'baroaltitude', 'velocity', 'vertrate'
        ),
    )
    if trajs is not None:
        file_date = start_date.strftime("%Y-%m-%d")
        trajs.to_parquet(os.path.join(parquet_dir_path, f"{file_date}.parquet"))
 
    # Update the start date
    start_date += timedelta(days=1)

In [3]:
## RETRIEVE FETCHED DATA FROM FOLDER ##
flights = []
folder_name = f"{month:02d}{year}"
folder_path = os.path.join("Flights/", folder_name)
for filename in os.listdir(folder_path):

    if filename.endswith(".parquet"):
        file_path = os.path.join(folder_path, filename)
        flight = Flight.from_file(file_path)
        flights.append(flight)

traff_set = Traffic.from_flights(flights)

#convert flights Timestamp to 'datetime64'
traff_set.data['timestamp'] = traff_set.data['timestamp'].dt.tz_convert(None).astype('datetime64[ns]')
traff_set.data['timestamp'] = traff_set.data['timestamp'].dt.tz_localize('UTC')

In [4]:
## FILTERING FLIGHTS ##
from traffic.core.mixins import PointMixin
extreme1 = PointMixin()
extreme2 = PointMixin()
extreme1.latitude = 47.5257
extreme1.longitude = 9.0068
extreme2.latitude = 47.5233
extreme2.longitude = 8.9996

center = PointMixin()
center.latitude = (extreme1.latitude + extreme2.latitude) / 2
center.longitude = (extreme1.longitude + extreme2.longitude) / 2

filtered_flights = []
numeric_columns = ["latitude", "longitude", "track", "altitude", "groundspeed", "vertical_rate"]

for flight in traff_set:
    if flight.data[numeric_columns].isna().all().all():
        continue
    
    flight.data[numeric_columns] = flight.data[numeric_columns].ffill().bfill()

    if any((flight.data[numeric_columns] == 0).all()):
        continue
    
    if flight.data.dropna(subset=numeric_columns).empty:
        continue

    dist = np.array(flight.distance(center).data['distance'])
    if np.min(dist) > 1:  # if minimum distance is greater than 1 NM (not even close to airport)
        continue 
    
    filtered_flights.append(flight)

traff_set_filtered = Traffic.from_flights(filtered_flights)
print(f"Total flights kept: {len(traff_set_filtered)}")

Total flights kept: 28


In [5]:
## ANALYZE FILTERED FLIGHTS AND CREATE DATA LIST ##
flight_data = []
for flight in traff_set_filtered:
    day = flight.data.timestamp.iloc[0].strftime("%d")
    
    lommis_func.analyze_flight(flight, airports["LSZT"], flight_data, [day, month, year], debug = False)

# sort columns by day and time
flight_data.sort(
    key=lambda row: (
        int(row[1]),
        datetime.strptime(row[4], "%H%M")
    )
)

In [6]:
## LOAD EXCEL TEMPLATE AND FILL IT OUT WITH NEW DATA ##
file_path = "Excel/template/template.xlsx"
wb = load_workbook(file_path)

ws_title = wb["TITLE"]

for row in ws_title.iter_rows():
    for cell in row:
        if cell.value == "Year":
            ws_title.cell(row=cell.row, column=cell.column + 2, value=year)
        elif cell.value == "Period":
            ws_title.cell(row=cell.row, column=cell.column + 2, value=month)

ws_data = wb["DATA"]

center_alignment = Alignment(horizontal="center", vertical="center")
for row in flight_data:
    ws_data.append(row)
    new_row_idx = ws_data.max_row
    for col_idx in range(1, len(row) + 1):
        cell = ws_data.cell(row=new_row_idx, column=col_idx)
        cell.alignment = center_alignment

new_filename = f"ARP_LSZT_{month:02d}{year}.xlsx"
new_file_path = os.path.join("Excel/", new_filename)

wb.save(new_file_path)
print(f"Modified Excel file saved as: {new_filename}")

Modified Excel file saved as: ARP_LSZT_012025.xlsx
